# **🧩 Overview of What We’ll Build**
A Python app that:

*   Takes a topic, article as input.
*   Generates Short Answer Questions.
*   Uses T5 fine-tuned models for question generation.
* Exports quizzes in structured format (JSON/CSV).

# **📦 Requirements**

In [1]:
!pip install transformers torch gradio newspaper3k nltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install lxml_html_clean

# **🧠 Python Code**

In [10]:
import gradio as gr
from transformers import T5ForConditionalGeneration, T5Tokenizer
from newspaper import Article
import nltk
import torch

nltk.download('punkt')

# Available models with descriptions
MODELS = {
    "base": {
        "name": "iarfmoose/t5-base-question-generator",
        "description": "General purpose question generator"
    },
    "squad": {
        "name": "mrm8488/t5-base-finetuned-question-generation-ap",
        "description": "Fine-tuned on SQuAD for educational questions"
    }
}

# Load tokenizer and model
current_model = "base"
tokenizer = T5Tokenizer.from_pretrained(MODELS[current_model]["name"])
model = T5ForConditionalGeneration.from_pretrained(MODELS[current_model]["name"])

def load_model(model_type):
    """Load the selected model"""
    global tokenizer, model, current_model
    if model_type != current_model:
        current_model = model_type
        tokenizer = T5Tokenizer.from_pretrained(MODELS[model_type]["name"])
        model = T5ForConditionalGeneration.from_pretrained(MODELS[model_type]["name"])
        torch.cuda.empty_cache()  # Clear cache if using GPU

def generate_questions(text, num_questions=5, model_type="base"):
    """Generate questions using the selected model"""
    # Load the selected model if different from current
    if model_type != current_model:
        load_model(model_type)

    input_text = f"generate questions: {text}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    outputs = model.generate(
        inputs,
        max_length=256,
        num_return_sequences=num_questions,
        do_sample=True,
        temperature=0.8,
        top_k=50,
        top_p=0.95,
        no_repeat_ngram_size=3  # Added to reduce repetition
    )

    questions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return questions

def quiz_generator(input_type, input_data, num_questions, model_type):
    text = input_data

    if not text.strip():
        return {"error": "No text provided or could not extract text from URL"}

    questions = generate_questions(text, num_questions, model_type)
    return {"questions": [{"question": q, "answer": "Short answer expected"} for q in questions]}

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# QuizWhiz AI")
    gr.Markdown("Generate Short Answer Questions from any article or text using T5.")

    with gr.Row():
        input_type = gr.Radio(choices=["Text"], label="Input Type", value="Text")
        num_questions = gr.Slider(minimum=1, maximum=10, step=1, value=5, label="Number of Questions")

    input_data = gr.Textbox(lines=10, label="Enter Text")

    with gr.Row():
        model_type = gr.Radio(
            choices=list(MODELS.keys()),
            label="Model Type",
            value="base",
            info="SQuAD model produces more educational questions"
        )
        model_info = gr.Markdown(MODELS["base"]["description"])

    output = gr.JSON(label="Generated Questions")

    # Update model info when selection changes
    def update_model_info(model_type):
        return MODELS[model_type]["description"]

    model_type.change(
        fn=update_model_info,
        inputs=model_type,
        outputs=model_info
    )

    submit = gr.Button("Generate Questions")
    submit.click(
        fn=quiz_generator,
        inputs=[input_type, input_data, num_questions, model_type],
        outputs=output
    )

if __name__ == "__main__":
    demo.launch()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://234c122ff8cd4c40ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## **Future Features Enchancements**
**Answer Generation:**

* Add functionality to generate suggested answers for each question

* Implement answer highlighting in the original text

**Multiple Question Types:**

* Support for different question formats (MCQ, True/False, Fill-in-the-blank)

* Option to specify desired question types

**Enhanced UI:**

* Question editing interface before final output

* Export options (JSON, CSV, PDF)
